# Goals
To visualize the the analyses of article data

# Libraries

In [152]:
from linearmodels.panel import PanelOLS, RandomEffects
from linearmodels.panel import compare as compare_models
import statsmodels.api as sm
import pandas as pd
import numpy as np
import os
import math
import csv
from io import StringIO
import pingouin as pg
import glob
from scipy.stats.mstats import zscore
from factor_analyzer import FactorAnalyzer

In [2]:
ROOT = os.path.dirname(os.getcwd()) + "/"

In [3]:
pd.set_option("mode.chained_assignment", None) 

# Read in Data

## JSTOR

In [4]:
articles = pd.read_feather(ROOT + "data/JSTOR/interim/cleaned.feather")
articles = articles[
    [
        "journal",
        "year",
        "volume",
        "issue",
        "title",
        "nwords",
        "npages",
        "nauthors",
        "auth1_name",
        "auth1_gender",
        "auth1_probwom",
        "emp",
        "gender_topic",
    ]
]
articles.sample(5)

,journal,year,volume,issue,title,nwords,npages,nauthors,auth1_name,auth1_gender,auth1_probwom,emp,gender_topic
231098,Health Education Research,2015,30,2,Evidence valued and used by health promotion p...,7602,13.0,3.0,None,None,NaN,12,0.000209
111487,Art Education,1994,47,2,Standards and Assessment: New Initiative and C...,3111,5.0,1.0,Jerome,man,0.00,1,0.000511
170352,The Journal of Education,1939,122,1,Spelling as a College Subject,1050,2.0,1.0,Alfred,man,0.00,0,0.001447
49976,Canadian Journal of Education / Revue canadien...,1991,16,3,"Le visiteur, le guide et l'éducation",2817,7.0,2.0,Bernard,man,0.01,0,0.000347
168849,The Journal of Education,1903,57,1 (1410),STATE EDUCATION,1443,2.0,1.0,None,None,NaN,0,0.001153


In [5]:
articles['emp'].describe()

count    243060.000000
mean          3.265198
std           8.815065
min           0.000000
25%           0.000000
50%           0.000000
75%           3.000000
max         440.000000
Name: emp, dtype: float64

In [6]:
articles['auth1_gender'].value_counts()

man      129451
woman     70400
Name: auth1_gender, dtype: int64

In [7]:
articles['gender_topic'].describe()

count    243060.000000
mean          0.007160
std           0.022911
min           0.000023
25%           0.000638
50%           0.001532
75%           0.004245
max           0.454584
Name: gender_topic, dtype: float64

## VDEM

In [8]:
vdem = pd.read_csv(ROOT + "/data/VDEM/V-Dem-CY-Full+Others-v12.csv", usecols=["year", "v2x_gencs", "v2x_gender"])
vdem["vdem"] = vdem.groupby("year")["v2x_gencs"].transform("mean")
vdem["vdem2"] = vdem.groupby("year")["v2x_gender"].transform("mean")
vdem = vdem.drop(["v2x_gencs", "v2x_gender"], axis=1)
vdem = vdem[(vdem["year"] > 1899)]
vdem = vdem.drop_duplicates()
vdem = vdem.astype({"year": int})
vdem = vdem.set_index('year')
vdem.sample(5)

,vdem,vdem2
year,,
1965,0.363596,0.415552
1995,0.620926,0.647462
1945,0.249980,0.285284
1953,0.305654,0.344493
1940,0.228527,0.262390


In [9]:
lag = 1
l_vdem = vdem.shift(lag).dropna()
print("unlagged start:", vdem.index.min())
print("lagged start:", l_vdem.index.min())

unlagged start: 1900
lagged start: 1901


In [10]:
max(vdem.index)

2021

## Merge

In [11]:
df = articles.merge(l_vdem, how="left", left_on="year", right_index=True)
df = df.sort_values(by=["journal", "year"])
df = df.reset_index(drop=True)
df['id'] = df.index
df = df.dropna(subset='vdem')
df.sample(5)

,journal,year,volume,issue,title,nwords,npages,nauthors,auth1_name,auth1_gender,auth1_probwom,emp,gender_topic,vdem,vdem2,id
4550,American Music Teacher,1993,43,3,Independent Music Teachers Forum,3529,2.0,4.0,Mary,woman,1.00,0,0.000453,0.594667,0.617420,4550
233592,The Science Teacher,1956,23,1,The Science Teacher and Problem Solving,2859,4.0,1.0,OREON,man,0.00,0,0.003676,0.307551,0.348750,233592
57051,Health Education Quarterly,1994,21,3,Ethnographic Approach to Community Organizatio...,4542,10.0,3.0,Ronald,man,0.00,8,0.000296,0.605269,0.628807,57051
66616,International Review of Education / Internatio...,1970,16,3,Teacher Training in Socialist Countries,4004,8.0,1.0,Dragutin,man,0.02,0,0.004786,0.376500,0.427711,66616
171256,The Journal of Education,1916,83,26 (2086),LANDSCAPE DEMONSTRATIONS BEING INTRODUCED IN R...,533,1.0,1.0,ELIZABETH,woman,1.00,0,0.006579,0.202993,0.237795,171256


In [12]:
df.shape

(243017, 16)

# Transform variables

In [167]:
ALL = ["title", "auth1_name", "volume", "journal",
       "gender_topic",
       "issue", 
       "nwords", 
       "year",
       "vdem",
       "vdem2",
       "auth1_gender", 
       "auth1_probwom",
       "emp", "id"]
data = df[ALL]

# Drop most recent 10 years
moving_wall = 10
data = data[data['year'] < data['year'].max()- moving_wall]


# When name unknwon, make it unknown
data['volume'].fillna("unknown", inplace=True)

# When name unknwon, make it unknown
data['auth1_name'].fillna("unknown", inplace=True)

# When gender NA, make it unknown
data['auth1_gender'].fillna("unknown", inplace=True)

# When prob unknown, make it chance
data['auth1_probwom'].fillna(.5, inplace=True)

# Drop extreme cases
data = data[(data['nwords']< np.percentile(data.nwords,99)) & (data['nwords'] > np.percentile(data.nwords,1))]

data = data.dropna()
data = data.reset_index(drop=True)

In [168]:
data.shape

(200292, 14)

In [186]:
data.year.min()

1910

In [169]:
# Issue becomes cat
data.issue = data.issue.astype('category').cat.codes

# Author dummies 
data["auth1_gender"] = data["auth1_gender"].fillna("unknown")
data.loc[
    (data["auth1_probwom"] >= 0.35) & (data["auth1_probwom"] <= 0.65), "auth1_gender"
] = "unknown"

data = data.join(pd.get_dummies(data["auth1_gender"]))
data['std_gender_topic'] = (data['gender_topic'] - data['gender_topic'].mean()) / data['gender_topic'].std()

# Standardize Vdem and empirical counts
data["std_vdem"] = (data["vdem"] - data["vdem"].mean()) / data["vdem"].std()
data["std_vdem2"] = (data["vdem2"] - data["vdem2"].mean()) / data["vdem2"].std()
data["std_emp"] = (data["emp"] - data["emp"].mean()) / data["emp"].std()

# Data-oriented articles
data["data_article"] = np.where((data["std_emp"] > 0), 1, 0)

In [170]:
data.shape

(200292, 22)

In [106]:
ALL = ['issue', 'nwords', 'year', 'std_vdem', 'std_vdem2',
       'man', 'unknown', 'woman', 'gender_topic', 'std_gender_topic',
       'data_article', 'auth1_probwom']

In [107]:
data.shape

(200292, 19)

In [108]:
data.to_csv(ROOT+"data/processed.csv")

## Table 1

In [171]:
def code_decade(year):
    cen = str(year // 100)
    dec = str(math.floor((year) % 100 / 10) * 10)
    if dec == "0":
        dec = "00"
    return cen + dec + "s"

In [172]:
data['decade'] = data['year'].apply(code_decade)

In [176]:
articles.columns

Index(['journal', 'year', 'volume', 'issue', 'title', 'nwords', 'npages',
       'nauthors', 'auth1_name', 'auth1_gender', 'auth1_probwom', 'emp',
       'gender_topic'],
      dtype='object')

In [177]:
articles['decade'] = articles['year'].apply(code_decade)

In [191]:
with open(ROOT+"reports/top10_all.csv", "w") as f: 
    writer = csv.writer(f)
    features = ["title", 
            "auth1_name",
            "journal",
            "year", 
            "volume", 
            "issue", 
            "auth1_probwom", 
            "gender_topic", 
            "emp"]
    writer.writerow(features)
    for decade, dec_df in articles.groupby("decade"):
        max_loading = sorted(dec_df['gender_topic'])[-1]
        row = []
        for feature in features: 
            feature = dec_df[dec_df["gender_topic"]==max_loading][feature].iloc[0]
            if type(feature) is np.float64:
                feature = round(feature, 3)
            row.append(feature)
        writer.writerow(row)
    
    # edge case: 2010
    only2010 = articles[articles['year']==2010]
    max_loading = only2010['gender_topic'].max()
    row = []
    for feature in features: 
        feature = only2010[only2010["gender_topic"]==max_loading][feature].iloc[0]
        if type(feature) is np.float64:
            feature = round(feature, 3)
        row.append(feature)    
    writer.writerow(row)

In [197]:
with open(ROOT+"reports/top10_ce.csv", "w") as f: 
    writer = csv.writer(f)
    features = ["title", 
            "auth1_name",
            "journal",
            "year", 
            "volume", 
            "issue", 
            "auth1_probwom", 
            "gender_topic", 
            "emp"]
    writer.writerow(features)
    ce = articles[articles['journal'] == "Comparative Education"]
    for decade, dec_df in ce.groupby("decade"):
        max_loading = sorted(dec_df['gender_topic'])[-1]
        row = []
        for feature in features: 
            feature = dec_df[dec_df["gender_topic"]==max_loading][feature].iloc[0]
            if type(feature) is np.float64:
                feature = round(feature, 3)
            row.append(feature)
        writer.writerow(row)
    
    # edge case: 2010
    only2010 = ce[ce['year']==2010]
    max_loading = only2010['gender_topic'].max()
    row = []
    for feature in features: 
        feature = only2010[only2010["gender_topic"]==max_loading][feature].iloc[0]
        if type(feature) is np.float64:
            feature = round(feature, 3)
        row.append(feature)    
    writer.writerow(row)

In [196]:
with open(ROOT+"reports/top10_cer.csv", "w") as f: 
    writer = csv.writer(f)
    features = ["title", 
            "auth1_name",
            "journal",
            "year", 
            "volume", 
            "issue", 
            "auth1_probwom", 
            "gender_topic", 
            "emp"]
    writer.writerow(features)
    ce = articles[articles['journal'] == "Comparative Education Review"]
    for decade, dec_df in ce.groupby("decade"):
        max_loading = sorted(dec_df['gender_topic'])[-1]
        row = []
        for feature in features: 
            feature = dec_df[dec_df["gender_topic"]==max_loading][feature].iloc[0]
            if type(feature) is np.float64:
                feature = round(feature, 3)
            row.append(feature)
        writer.writerow(row)
    
    # edge case: 2010
    only2010 = ce[ce['year']==2010]
    max_loading = only2010['gender_topic'].max()
    row = []
    for feature in features: 
        feature = only2010[only2010["gender_topic"]==max_loading][feature].iloc[0]
        if type(feature) is np.float64:
            feature = round(feature, 3)
        row.append(feature)    
    writer.writerow(row)

In [198]:
data['gender_topic'].describe()

count    200292.000000
mean          0.007189
std           0.023407
min           0.000061
25%           0.000661
50%           0.001522
75%           0.004058
max           0.454584
Name: gender_topic, dtype: float64

## Table 2a

In [109]:
data.to_csv(ROOT+"data/stat_data.csv")

In [110]:
table1 = data
table1 = table1.set_index(['journal', 'id'])[ALL]
table1.agg(['count', 'mean', 'std', 'min', 'max']).T.astype(float).round(2).to_excel(ROOT+"reports/table_1a.xlsx", 
                        index=True, 
                        engine='xlsxwriter', 
                        sheet_name=f'table_2a')

## Table 2b

In [111]:
pd.DataFrame(np.tril(table1[ALL].corr().round(2).to_numpy()), 
             columns=ALL,
             index=ALL).to_excel(ROOT+"reports/table_1b.xlsx", 
                        index=True, 
                        engine='xlsxwriter',
                        sheet_name=f'table_2b')

# Model

Compute interaction terms

In [112]:
data['probXvdem2'] = data['auth1_probwom'] * data['std_vdem2']

In [113]:
data['womanXyear'] = data['woman'] * data['year']
data['manXyear'] = data['man'] * data['year']

In [114]:
def write_table(path, results, table_n):
    '''
    takes a path, PanelOLS results obj, and table number
    transforms PanelOLS results object into a pandas df
    writes an excel of results at specified path
    '''
    results_obj = compare_models(results, precision='std_errors', stars=True)
    results_csv = results_obj.summary.as_csv()
    results_pth = StringIO(results_csv)
    results_df = pd.read_csv(results_pth, skiprows=1, engine='python')
    results_df.drop(results_df.index[[10]]).to_excel(path+f'table_{table_n}.xlsx', 
                        index=False, 
                        engine='xlsxwriter', 
                        sheet_name=f'table_{table_n}')

In [115]:
def nest_models(controls):
    '''
    takes a baseline model;
    returns taxonomy of nested models
    with m3 modifiable
    '''
    ms = []
    #0
    ms.append(controls)
    #1
    ms.append(controls + ['year'])
    #2
    ms.append(controls + ['std_vdem2'])
    #3
    ms.append(ms[2] + ['auth1_probwom'])
    #4
    ms.append(ms[3] + ['probXvdem2'])
    #5
    ms.append(ms[2] + ['data_article'])
    #6
    ms.append(ms[4] + ['data_article'])
    return ms

## Table 3

In [117]:
panel_data = data.set_index(["journal", "id"])
outcome = panel_data.std_gender_topic
models = nest_models(controls=["issue", "nwords"])
results = {}
for i, model in enumerate(models):
    predictors = sm.add_constant(panel_data[model])
    model = PanelOLS(outcome, predictors, entity_effects=True, check_rank=True, drop_absorbed=True)
    res = model.fit(cov_type='clustered')
    if f"Model {i}" not in results:
        results[f"Model {i}"] = res
write_table(path=ROOT+'reports/', results=results, table_n="3")

In [118]:
data['journal'].nunique()

162

## Table 4

In [119]:
CER = "Comparative Education Review"
CE = "Comparative Education"

In [120]:
ols_data = data.copy()
ols_data = ols_data[(ols_data["journal"] == CER) | (ols_data["journal"] == CE)]
ols_data = ols_data.set_index(["journal", "id"])
outcome = ols_data.std_gender_topic
models = nest_models(controls=["issue", "nwords"])
results = {}
for i, model in enumerate(models):
    predictors = sm.add_constant(ols_data[model])
    model = PanelOLS(outcome, predictors, entity_effects=True)
    res = model.fit(cov_type='clustered')
    if f"Model {i}" not in results:
        results[f"Model {i}"] = res
write_table(path=ROOT+'reports/', results=results, table_n="4")

# Robustness checks

## Binary gender
How robust are our results when we represent women as a binary? 

In [78]:
data['womanXvdem2'] = data['woman'] * data['std_vdem2']

In [79]:
def nest_models(controls):
    '''
    takes a baseline model;
    returns taxonomy of nested models
    with m3 modifiable
    '''
    ms = []
    #0
    ms.append(controls)
    #1
    ms.append(controls + ['year'])
    #2
    ms.append(controls + ['std_vdem2'])
    #3
    ms.append(ms[2] + ['woman'])
    #4
    ms.append(ms[3] + ['womanXvdem2'])
    #5
    ms.append(ms[2] + ['data_article'])
    #6
    ms.append(ms[4] + ['data_article'])
    return ms

### Table D1

In [80]:
panel_data = data.set_index(["journal", "id"])
outcome = panel_data.std_gender_topic
models = nest_models(controls=["issue", "nwords"])
results = {}
for i, model in enumerate(models):
    predictors = sm.add_constant(panel_data[model])
    model = PanelOLS(outcome, predictors, entity_effects=True, check_rank=True, drop_absorbed=True)
    res = model.fit(cov_type='clustered')
    if f"Model {i}" not in results:
        results[f"Model {i}"] = res
write_table(path=ROOT+'reports/', results=results, table_n="d1")

### Table D2

In [81]:
ols_data = data.copy()
ols_data = ols_data[(ols_data["journal"] == CER) | (ols_data["journal"] == CE)]
ols_data = ols_data.set_index(["journal", "id"])
outcome = ols_data.std_gender_topic
models = nest_models(controls=["issue", "nwords"])
results = {}
for i, model in enumerate(models):
    predictors = sm.add_constant(ols_data[model])
    model = PanelOLS(outcome, predictors, entity_effects=True)
    res = model.fit(cov_type='clustered')
    if f"Model {i}" not in results:
        results[f"Model {i}"] = res
write_table(path=ROOT+'reports/', results=results, table_n="d2")

In [130]:
ols_data[['auth1_probwom', 'vdem2']].corr()

,auth1_probwom,vdem2
auth1_probwom,1.000000,0.207297
vdem2,0.207297,1.000000


## Civil Society Participation
How robust are our results when we more narrowly represent women's empowerment as an index of their civil society participation? 

In [82]:
data['probXvdem'] = data['auth1_probwom'] * data['std_vdem']

In [83]:
def nest_models(controls):
    '''
    takes a baseline model;
    returns taxonomy of nested models
    with m3 modifiable
    '''
    ms = []
    #0
    ms.append(controls)
    #1
    ms.append(controls + ['year'])
    #2
    ms.append(controls + ['std_vdem'])
    #3
    ms.append(ms[2] + ['auth1_probwom'])
    #4
    ms.append(ms[3] + ['probXvdem'])
    #5
    ms.append(ms[2] + ['data_article'])
    #6
    ms.append(ms[4] + ['data_article'])
    return ms

### Table E1

In [84]:
panel_data = data.set_index(["journal", "id"])
outcome = panel_data.std_gender_topic
models = nest_models(controls=["issue", "nwords"])
results = {}
for i, model in enumerate(models):
    predictors = sm.add_constant(panel_data[model])
    model = PanelOLS(outcome, predictors, entity_effects=True, check_rank=True, drop_absorbed=True)
    res = model.fit(cov_type='clustered')
    if f"Model {i}" not in results:
        results[f"Model {i}"] = res
write_table(path=ROOT+'reports/', results=results, table_n="e1")

### Table E2

In [85]:
ols_data = data.copy()
ols_data = ols_data[(ols_data["journal"] == CER) | (ols_data["journal"] == CE)]
ols_data = ols_data.set_index(["journal", "id"])
outcome = ols_data.std_gender_topic
models = nest_models(controls=["issue", "nwords"])
results = {}
for i, model in enumerate(models):
    predictors = sm.add_constant(ols_data[model])
    model = PanelOLS(outcome, predictors, entity_effects=True)
    res = model.fit(cov_type='clustered')
    if f"Model {i}" not in results:
        results[f"Model {i}"] = res
write_table(path=ROOT+'reports/', results=results, table_n="e2")

## Lagged Periods
How robust are the institutional effects when we try different lags? Below is a rinse and repeat of all the data prep and modeling in one cell, wrapped into a loop of different lags. 

In [101]:
results = {}
lags = [1,3,5,10]
for lag in lags: 
    
    #------------------------------------#
    # CREATE MAIN DATA SET, LAGGED
    #------------------------------------#
    
    # Lag
    l_vdem = vdem.shift(lag).dropna()
    
    # Merge
    df = articles.merge(l_vdem, how="left", left_on="year", right_index=True)
    df = df.sort_values(by=["journal", "year"])
    df = df.reset_index(drop=True)
    df['id'] = df.index
    df = df.dropna(subset='vdem')
    
    # Subset only vars we care about
    ALL = ["journal",
           "gender_topic",
           "issue", 
           "nwords", 
           "year",
           "vdem",
           "vdem2",
           "auth1_gender", 
           "auth1_probwom",
           "emp", "id"]
    data = df[ALL]
    
    
    #------------------------------------#
    # TRANSFORM VARIABLES 
    #------------------------------------#

    # Drop most recent 10 years
    moving_wall = 10
    data = data[data['year'] < data['year'].max()- moving_wall]

    # When gender NA, make it unknown
    data['auth1_gender'].fillna("unknown", inplace=True)

    # When prob unknown, make it chance
    data['auth1_probwom'].fillna(.5, inplace=True)

    # Drop extreme cases
    data = data[(data['nwords']< np.percentile(data.nwords,99)) & (data['nwords'] > np.percentile(data.nwords,1))]

    data = data.dropna()
    data = data.reset_index(drop=True)

    # Issue becomes cat
    data.issue = data.issue.astype('category').cat.codes

    # Author dummies 
    data["auth1_gender"] = data["auth1_gender"].fillna("unknown")
    data.loc[
        (data["auth1_probwom"] >= 0.35) & (data["auth1_probwom"] <= 0.65), "auth1_gender"
    ] = "unknown"

    data = data.join(pd.get_dummies(data["auth1_gender"]))
    data['std_gender_topic'] = (data['gender_topic'] - data['gender_topic'].mean()) / data['gender_topic'].std()

    # Standardize Vdem and empirical counts
    data["std_vdem"] = (data["vdem"] - data["vdem"].mean()) / data["vdem"].std()
    data["std_vdem2"] = (data["vdem2"] - data["vdem2"].mean()) / data["vdem2"].std()
    data["std_emp"] = (data["emp"] - data["emp"].mean()) / data["emp"].std()

    # Data-oriented articles
    data["data_article"] = np.where((data["std_emp"] > 0), 1, 0)


    #------------------------------------#
    # RE-FIT MODEL 6 
    #------------------------------------#
    
    # Predictors
    data['probXvdem2'] = data['auth1_probwom'] * data['std_vdem2']
    model_6 = ['issue', 
               'nwords', 
               'std_vdem2', 
               'auth1_probwom', 
               'probXvdem2', 
               'data_article']
    
    # Model Full Data
    print(f"n journals for lag {lag}: {data['journal'].nunique()}")
    panel_data = data.set_index(["journal", "id"])
    outcome = panel_data.std_gender_topic
    predictors = sm.add_constant(panel_data[model_6])
    model = PanelOLS(outcome, predictors, entity_effects=True, check_rank=True, drop_absorbed=True)
    results[f'Model 6 all_{lag}'] = model.fit(cov_type='clustered')
    
    
    # Model CER Data 
    ols_data = data.copy()
    ols_data = ols_data[(ols_data["journal"] == CER) | (ols_data["journal"] == CE)]
    ols_data = ols_data.set_index(["journal", "id"])
    outcome = ols_data.std_gender_topic
    predictors = sm.add_constant(ols_data[model_6])
    model = PanelOLS(outcome, predictors, entity_effects=True)
    results[f'Model 6 cer_{lag}'] = model.fit(cov_type='clustered')

write_table(path=ROOT+'reports/', results=results, table_n="f1")

n journals for lag 1: 163
n journals for lag 3: 162
n journals for lag 5: 162
n journals for lag 10: 162
